In [55]:
from typing import List, Callable, Tuple # potrzebne do utworzenia genomu, FitnessFunc (https://docs.python.org/3/library/typing.html#typing.Callable)
from random import choices, randint, randrange, random # potrzebne do losowań
from collections import namedtuple # potrzebne do utworzenia listy przedmiotów
from functools import partial # potrzebne do częściowego wykorzystywania funkcji
import time # potrzebne do pomiaru czasu działania algorytmu

In [74]:
Genome = List[int] # Genom - lista przedmiotów określona binarnie
Population = List[Genome] # Populacja - lista list przedmiotów (genomów)
Thing = namedtuple('Thing', ['name', 'value', 'weight']) # namedtuple() jest specjalnym Pythonowym obiektem, podobnym do słownika

# w celu rozdzielenia problemu od algorytmu (charakterystyka metaheurystyk), będziemy wykorzystywać dodatkowe funkcje
FitnessFunc = Callable[[Genome], int] # funkcja dopasowania - składnia Callable() musi być zawsze używana z dokładnie dwiema wartościami: listą argumentów i typem zwracanym.
PopulationFunc = Callable[[], Population] # nie przyjmuje wartości, a zwraca populacje
SelectionFunction = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]] # wybiera dwa rozwiązania w celu ich krzyżowania (rodzice -> dziecko)
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]] # bierze dwa genomy i zwraca nowy genom
MutationFunc = Callable[[Genome], Genome] # bierze jeden genom i zwraca nowy genom

things = [
    Thing('Laptop', 500, 2200),
    Thing('Headphones', 150, 160),
    Thing('Coffe Mug', 60, 350),
    Thing('Notepad', 40, 333),
    Thing('Water Bottle', 30, 192),
]

more_things = [
    Thing('Mints', 5, 25),
    Thing('Socks', 10, 38),
    Thing('Tissues', 15, 80),
    Thing('Phone', 500, 200),
    Thing('Baseball Cap', 100, 70),
] + things

def generate_genome(length: int) -> Genome:
    # "->" oznacza adnotacje do funkcji, mówi ona "third-party" programom oraz nam, jaki typ zwraca dana funkcja
    # jest to nieobligatoryjny zapis
    # https://www.youtube.com/watch?v=k56rEoRjK4k
    # https://peps.python.org/pep-3107/  
    # https://renenyffenegger.ch/notes/development/languages/Python/dunders/__annotations__/index
    return choices([0,1], k = length)

def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)] # podkreślnik "_" jest "poprawniejszym" zapisem w języku iteracji (miast pisania i,j,k,l - jak w matematyce)

def fitness(genome: Genome, things: [Thing], weight_limit: int) -> int:
    # odwołania typu "genome: Genome" służą do wykorzystania wczesniej utworzonych obiektów
    if len(genome) != len(things):
        raise ValueError("genome and things must be of the same length")
        # jest to niezbędne, w przeciwnym wypadku tablica przedmiotów i tablica genomów [1,0,0,0,1 ...] itp. nie będą się pokrywać
        
    weight = 0
    value = 0
    
    for i, thing in enumerate(things): # enumerate() - ułatwia "przechodzenie" po przedmiotach z listy https://realpython.com/python-enumerate/
        if genome[i] == 1:
            weight += thing.weight
            value += thing.value
            # sepcjalna własność (odwoływań) funkcji enumerate()
            
            if weight > weight_limit: # jeżeli przekroczymy limit wagowy, to rozwiązanie "do wyrzucenia"
                return 0
            
    return value

def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    # Dlaczego nie odwołujemy się do naszej funkcji fitness?
    # Jest to związane z architekturą oprogramowania oraz rozdzieleniem problemów.
    # Wcześniej napisana funkcja jest dedykowana problemowi pakowania plecaka (dodatkowe dwa argumenty - rzeczy, dopuszczalna waga)
    return choices(
        population = population,
        weights = [fitness_func(genome) for genome in population],
        # przez wykorzystanie wag prawdopodobieństwa, podpowiadamy funkcji, aby genomy o większej wadzę wybierał z większym prawdopodobieństwem
        k = 2
    )

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("genome and things must be of the same length")
        
    # jeżeli obie listy genomów będą długości 1, to nic sobie nie pomieszamy
    length = len(a)
    if length < 2:
        return a, b
    
    # mieszanie dwóch genomów w losowym ich "przecięciu"
    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome: # z pewnym prawdopodobieństwem będziemy zmieniać 0 -> 1 oraz 1 -> 0
    for _ in range(num):
        index = randrange(len(genome)) # mniej kosztowna funkcja losująca indeks z określonego zakresu
        # https://stackoverflow.com/questions/3540431/what-is-the-difference-between-random-randint-and-randrange
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
        # inna konstrukcja warunku "if" tj. w przypadku wylosowania liczby większej od probability, przypisuje genome[index] = genome[index]
        # w przeciwnym wypadku abs(genome[index] - 1) - nalezy rozważyć oba możliwe przypadki:
        # jeżeli genome[index] = 1, to abs(genome[index] - 1) = abs(1 - 1) = 0
        # jeżeli genome[index] = 0, to abs(genome[index] - 1) = abs(0 - 1) = abs(-1) = 1
        # sprytna zamiana wartości
    return genome

def run_evolution(
    populate_func: PopulationFunc,
    fitness_func: FitnessFunc,
    fitness_limit: int, # warunek końcowy dla dopasowania funkcji fitness
    selection_func: SelectionFunction = selection_pair,
    crossover_func: CrossoverFunc = single_point_crossover,
    mutation_func: MutationFunc = mutation,
    generation_limit: int = 100 # warunek końcowy dla liczby pokoleń
) -> Tuple[Population, int]:
    population = populate_func() # wskazanie populacji
    
    for i in range(generation_limit): # w tym wypadku, sortowanie stale zmieni kolejność w danych
        population = sorted(
            population,
            key = lambda genome: fitness_func(genome), # sortowanie względem wyniku funkcji fitness na genomie
            reverse = True
        )
        
        if fitness_func(population[0]) >= fitness_limit: # jeżeli pierwszy (najlepszy) genom jest lepszy od wymaganego limitu, to stop
            break
            
        next_generation = population[0:2] # zostawiamy sobie top dwa rozwiązania dla naszej kolejnego pokolenia
        
        # utworzenie nowych genomów dla kolejnych pokoleń
        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func) # wybieramy parę (możliwie najlepszą)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1]) # krzyżujemy parę rodziców
            offspring_a = mutation_func(offspring_a) # mutujemy pierwsze dziecko
            offspring_b = mutation_func(offspring_b) # mutujemy drugie dziecko
            next_generation += [offspring_a, offspring_b] # dodajemy kolejne zmutowane dzieci do nowego pokolenia
        
        population = next_generation # nadpisanie starego pokolenia, nowym
        
    # po osiągnieciu fitness_limit lub generation_limit, musimy zwrócić jakąś wartość, stąd ponowne sortowanie w przypadku osiągnięcia limitu pokoleń
    population = sorted(
        population,
        key = lambda genome: fitness_func(genome),
        reverse = True
    )
    
    # zwracamy populację oraz liczbe iteracji
    return population, i

start = time.time()
# wywołanie algorytmu - skorzystamy z funkcji partial(), która potrafi ograniczyć wprowadzane zmienne, poprzez ustawienie pozsotałych na stałe (określone przez nas)
# https://www.geeksforgeeks.org/partial-functions-python/
population, generations = run_evolution(
    populate_func = partial(
        generate_population, size = 10, genome_length = len(things)
    ),
    fitness_func = partial(
        fitness, things = things, weight_limit = 3000
    ),
    fitness_limit = 740,
    generation_limit = 100
)
end = time.time()

# funkcja, która pomaga w wizualizacji print-a (nieobligatoryjna)
def genome_to_things(genome: Genome, things: [Thing]) -> [Thing]:
    result = []
    for i, thing in enumerate(things):
        if genome[i] == 1:
            result += [thing.name]
    
    return result

print(f"number of generations: {generations}")
print(f"time: {end - start}s")
print(f"best solution: {genome_to_things(population[0], things)}")

number of generations: 5
time: 0.0010120868682861328s
best solution: ['Laptop', 'Headphones', 'Coffe Mug', 'Water Bottle']


In [75]:
things

[Thing(name='Laptop', value=500, weight=2200),
 Thing(name='Headphones', value=150, weight=160),
 Thing(name='Coffe Mug', value=60, weight=350),
 Thing(name='Notepad', value=40, weight=333),
 Thing(name='Water Bottle', value=30, weight=192)]

In [77]:
tst_lab = {1: [2, 3, 4], 2: [1, 7, 6], 3: [1, 6], 4: [1, 5], 5: [4], 6: [2, 3], 7: [2]}
tst_lab[1][1]

3